# Structured Output with Groq and Instructor
While large language models (LLMs) are often employed for building chatbots or conversational agents, numerous real-world applications require a different approach - one that goes beyond mere dialogue and involves producing structured, machine-readable outputs.

Consider a typical scenario: we want to produce structured JSON data from an LLM. While tools like Python's `json` module allow us to handle this data, they also come with their own set of challenges, such as validating data types and ensuring consistency across outputs. Manually checking these aspects can be tedious and error-prone. LLMs also tend to forget to include a comma or a closing bracket ('}') somewhere in the produced JSON from time to time, which would invalidate the whole JSON output.

This is where the Instructor library comes into play. If you've been using the recent feature of [structured outputs](https://openai.com/index/introducing-structured-outputs-in-the-api/) by OpenAI, then you'll feel at home using Instructor. By integrating the Instructor library with models powered by Groq, we can simplify the process of generating structured outputs, making it both easier and more reliable. This guide will walk you through setting up this integration, showcasing how to use structured outputs to generate synthetic data for evaluating LLM-powered applications - a powerful and practical use case for LLMs.

## 1. A Very Simple Use Case

Let's dive right into how you can set up the `instructor` library with Groq models to generate structured JSON outputs. We'll keep it simple and straightforward so you can get up and running quickly.

### Grabbing Your API Key

Before doing anything, grab your Groq API key from [Groq console](https://console.groq.com/keys) and put it in an `.env` file alongside this notebook:
```bash
GROQ_API_KEY=<YOUR_API_KEY>
```

### Installing the Necessary Libraries

Install the required Python libraries. You'll need:
- groq
- instructor
- python-dotenv (for loading environment variables)

Run the following command to install the libraries:
```bash
pip install -U groq instructor python-dotenv
```

### Extracting Structured Data

Let's consider a very simple use case. Imagine you want to extract user details like name, age, and email from a piece of text that you have. To extract this information, you could utilize a JSON schema to define the structure of the user data and pass it to a Groq model (for example, by passing `response_format={"type": "json_object"}` to the `chat.completions.create()` method). However, creating JSON schemas can be cumbersome. To facilitate this, instructor leverages the Pydantic library, a powerful tool that simplifies the process of describing the output structure of the model:

In [1]:
import instructor
from dotenv import load_dotenv
from pydantic import BaseModel
from groq import Groq

# Load the Groq API key from .env file
load_dotenv()

# Describe the desired output schema using pydantic models
class UserInfo(BaseModel):
    name: str
    age: int
    email: str

# The text to extract data from
text = """
John Doe, a 35-year-old software engineer from New York, has been working with large language models for several years.
His email address is johndoe@example.com.
"""

# Patch Groq() with instructor, this is where the magic happens!
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

# Call the API
user_info = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    response_model=UserInfo, # Specify the response model
    messages=[
        {"role": "system", "content": "Your job is to extract user information from the given text."},
        {"role": "user", "content": text}
    ],
    temperature=0.65,
)

print(f"Name: {user_info.name}")
print(f"Age: {user_info.age}")
print(f"Email: {user_info.email}")

Name: John Doe
Age: 35
Email: johndoe@example.com


In the example above, we've defined a simple pydantic model `UserInfo` that specifies a person's name (as a string), age (as an integer), and email (as a string). The `instructor` library ensures that the Groq model's output adheres to this schema. The great thing here is that the `instructor` library ensures the response is valid according to the schema you provided. This eliminates the need for manual validation and reduces the likelihood of errors creeping into your data.

## 2. A More Serious Use Case: Generating Synthetic Data

Imagine you are designing a weather agent capable of calling functions (tools). This agent is given a `get_weather_info` tool to retrieve the latest weather information about a location. The JSON schema for this tool is provided here:

```json
{
    "name": "get_weather_info",
    "description": "Get the weather information for any location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location for which we want to get the weather information (e.g., New York)" 
            }
        },
        "required": ["location"]
    }
}
```

Our goal is to create a structured dataset of realistic examples that simulate how a user might request weather information in various scenarios. We want to use a large language model (LLM) to generate these examples for us and use them as an evaluation set to test our agent's capabilities. Without such an evaluation, we lack a way to understand the effects of our prompt adjustments. These examples will not only help us evaluate the agent's ability to use the `get_weather_info` tool correctly but also make it easy to detect if any prompt changes have negative effects.

Now, let's use the `instructor` library with Groq to generate synthetic examples for our weather agent.

### Defining the Task and Schema

To generate these examples, we need to write a prompt that instructs the model to create scenarios where an agent would use the `get_weather_info` tool. We can use the following system prompt for this task:

In [2]:
from pprint import pprint

import instructor
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from groq import Groq

# Load the Groq API key from .env file
load_dotenv()

prompt = """
I am designing a weather agent. This agent can talk to the user and also fetch latest weather information.
It has access to the `get_weather_info` tool with the following JSON schema:
{json_schema}

I want you to write some examples for `get_weather_info` and see if this functionality works correctly and can handle all the cases. 
Now given the information so far and the JSON schema of the provided tool, write {num} examples.
Make sure each example is varied enough to cover common ways of requesting for this functionality.
Make sure you fill the function parameters with the correct types when generating the output examples. 
Make sure your output is valid JSON.
"""

We now need to specify the structure of the output. For this task, I want the output to include the example text, the tool to call, and also the parameters of the tool. Something like the following:
```json
{
    "examples": [
        {
            "input_text": "Get the weather information for San Francisco.",
            "tool_name": "get_weather_info",
            "tool_parameters": "{\"location\":\"San Francisco\"}"
        },
        ...
    ]
}
```
We can easily translate this structure into a Pydantic model like the following:

In [3]:
class Example(BaseModel):
    input_text: str = Field(description="The example text")
    tool_name: str = Field(description="The tool name to call for this example")
    tool_parameters: str = Field(description="An object containing the key-value pairs for the parameters of this tool as a JSON serializbale STRING, make sure it is valid JSON and parameter values are of the correct type according to the tool schema")

class ResponseModel(BaseModel):
    examples: list[Example]

### Generating the Examples
Now let's call the Groq API with our custom prompt and ask it to generate 5 examples for us:

In [4]:
# The schema for get_weather_info tool
tool_schema = {
    "name": "get_weather_info",
    "description": "Get the weather information for any location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location for which we want to get the weather information (e.g. New York)"
            }
        },
        "required": ["location"]
    }
}

# Patch Groq() with instructor, this is where the magic happens!
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

# Call the API with our custom prompt and ResponseModel
response = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    response_model=ResponseModel, # Specify the response model
    messages=[
        {
            "role": "system", 
            "content": prompt.format(json_schema=tool_schema, num=5), # Pass the tool schema and number of examples to the prompt
        },
    ],
    temperature=0.65,
    max_tokens=8000,
)

print(type(response))
pprint(response.examples)

<class '__main__.ResponseModel'>
[Example(input_text="What's the weather like in New York?", tool_name='get_weather_info', tool_parameters='{"location": "New York"}'),
 Example(input_text='Get the weather forecast for London', tool_name='get_weather_info', tool_parameters='{"location": "London"}'),
 Example(input_text='I want to know the weather in Paris', tool_name='get_weather_info', tool_parameters='{"location": "Paris"}'),
 Example(input_text='What is the weather like in Tokyo, Japan?', tool_name='get_weather_info', tool_parameters='{"location": "Tokyo, Japan"}'),
 Example(input_text='Can you tell me the weather in Sydney, Australia?', tool_name='get_weather_info', tool_parameters='{"location": "Sydney, Australia"}')]


As you can see, the model returned an object of type `ResponseModel` and has correctly created 5 examples for our evaluation dataset. Great!

## Conclusion

With these generated examples, you can evaluate how well the weather agent utilizes the `get_weather_info` tool. By simulating various scenarios, you can test the agent's ability to handle different contexts, ensuring that it correctly identifies when and how to call the tool with the appropriate parameters. This approach enhances not only the evaluation process but also helps identify edge cases where the agent might struggle, allowing you to refine its performance before deploying it in a real-world environment.

This notebook demonstrates just one of the powerful use cases of structured outputs in Large Language Models (LLMs). By combining the `instructor` library with Groq, you can effortlessly create a diverse set of structured examples, making your agentic workflows more robust and reliable.

### Useful Links
- [Groq Cookbook](https://github.com/groq/groq-api-cookbook)
- [Instructor Docs](https://python.useinstructor.com/)
- [OpenAI Structured Output Blog](https://openai.com/index/introducing-structured-outputs-in-the-api/)
